In [24]:
#Importing the necessary packages
import cv2 as cv
import numpy as np
import dlib
import torch
import matplotlib.pyplot as plt
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread 
import time
import argparse
import playsound
import imutils

In [25]:
import multiprocessing
from playsound import playsound

p = multiprocessing.Process(target=playsound, args=("beep.wav",))
p.start()
input("press ENTER to stop playback")
p.terminate()

In [26]:
#plays Alarm sound
def alarm():
  playsound("beep.wav")

def EAR(eye):
  a=np.linalg.norm(eye[1]-eye[5])
  b=np.linalg.norm(eye[2]-eye[4])
  c=np.linalg.norm(eye[0]-eye[3])

  ear=(a+b)/(2*c)
  return ear

def MAR(mouth):
  h1=np.linalg.norm(mouth[13]-mouth[19])
  h2=np.linalg.norm(mouth[14]-mouth[18])
  h3=np.linalg.norm(mouth[15]-mouth[17])
  w=np.linalg.norm(mouth[12]-mouth[16])
  mar=(h1+h2+h3)/(3*w)
  
  return mar
  

In [27]:
# #%%writefile parsing.py
# parser=argparse.ArgumentParser()
# parser.add_argument("-p", '--shape predictor', required=True, help="path to facial landmark predictor")
# parser.add_argument("-a", "--alarm", type=str, default="",help="path alarm .WAV file")
# parser.add_argument("-w", "--webcam", type=int, default=0,help="index of webcam on system")
# args = vars(parser.parse_args())

In [41]:
#Eye
EAR_threshold=.32
frames_threshold=20

#Mouth
MAR_threshold=.15
Mouth_frames_threshold=20

COUNTER=0
COUNTER_yawn=0
alarm_status=False
alarm_status_yawn=False

In [29]:
#Initialising the Dlib's face detector (which is based on HOG)
landmark_det=dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [30]:
(left_eye_start, left_eye_end)=face_utils.FACIAL_LANDMARKS_IDXS['left_eye']
(right_eye_start, right_eye_end)=face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [31]:
(mouth_start, mouth_end)=face_utils.FACIAL_LANDMARKS_IDXS['mouth']

In [32]:
photo=cv.imread('PA030984.JPG.jpg')
gray = cv.cvtColor(photo, cv.COLOR_BGR2GRAY)

In [33]:
photo=imutils.resize(photo, width=500)

In [34]:
rects=landmark_det(photo, 1)
my_shape = predictor(gray, rects[0])
my_shape = face_utils.shape_to_np(my_shape)

In [35]:
My_mouth=my_shape[mouth_start: mouth_end]

In [36]:
My_mouth[0:7]

array([[244, 113],
       [249, 110],
       [254, 109],
       [258, 110],
       [261, 109],
       [266, 110],
       [271, 112]])

In [37]:

def convert_and_trim_bb(image, rect):
	# extract the starting and ending (x, y)-coordinates of the
	# bounding box
	startX = rect.left()
	startY = rect.top()
	endX = rect.right()
	endY = rect.bottom()
	# ensure the bounding box coordinates fall within the spatial
	# dimensions of the image
	startX = max(0, startX)
	startY = max(0, startY)
	endX = min(endX, image.shape[1])
	endY = min(endY, image.shape[0])
	# compute the width and height of the bounding box
	w = endX - startX
	h = endY - startY
	# return our bounding box coordinates
	return (startX, startY, w, h)

boxes = [convert_and_trim_bb(photo, r) for r in landmarks_pos]
#cv.rectangle(gray,r.left(),r.top(), r.right(), r.top(), r.bottom() )

NameError: name 'landmarks_pos' is not defined

In [ ]:
boxes

[(221, 61, 72, 72)]

In [ ]:
for (x, y, w, h) in boxes:
	# draw the bounding box on our image
	cv.rectangle(photo, (x, y), (x + w, y + h), (0, 255, 0), 2)

cv.imshow("Output", photo)
cv.waitKey(0)

48

In [ ]:
dlib.rectangle(rect[0].left(), rect[0].top(), rect[0].right(),rect[0].bottom())

rectangle(221,61,293,133)

In [42]:
cap=cv.VideoCapture(0)
while True:
	ret, frame=cap.read()
	frame = imutils.resize(frame, width=600)
	gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
	rects = landmark_det(frame, 1)
	for rect in rects:
		# determine the facial landmarks for the face region, then
		# convert the facial landmark (x, y)-coordinates to a NumPy
		# array
		#face_rectangle = dlib.rectangle(rect.left(), rect.top(), rect.right(),rect.bottom())
		cv.rectangle(frame, (rect.left(), rect.top()), (rect.right(),rect.bottom()), (0,0,255),2)
		cv.putText(frame, "Hiii!", (rect.left() - 10, rect.top() - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
		my_shape = predictor(gray, rect)
		my_shape = face_utils.shape_to_np(my_shape)
		# extract the left and right eye coordinates, then use the
		# coordinates to compute the eye aspect ratio for both eyes
		leftEye = my_shape[left_eye_start:left_eye_end]
		rightEye = my_shape[right_eye_start:right_eye_end]
		leftEAR = EAR(leftEye)
		rightEAR = EAR(rightEye)
		# average the eye aspect ratio together for both eyes
		ear_tot = (leftEAR + rightEAR) / 2.0
		leftEyeHull = cv.convexHull(leftEye)
		rightEyeHull = cv.convexHull(rightEye)
		cv.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
		cv.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

		My_mouth=my_shape[mouth_start: mouth_end]
		My_mouth_MAR=MAR(My_mouth)
		cv.drawContours(frame, [My_mouth], -1, (0, 255, 0), 1)
	
		#Do the same thing for mouth

		if ear_tot < EAR_threshold:
			COUNTER += 1
			# if the eyes were closed for a sufficient number of
			# then sound the alarm
			if COUNTER >= frames_threshold:
				# if the alarm is not on, turn it on
				if not alarm_status:
					alarm_status = True
					# check to see if an alarm file was supplied,
					# and if so, start a thread to have the alarm
					# sound played in the background
					#if args["alarm"] != "":
					alarm()
					t = Thread(target=alarm)
					t.deamon = True
					t.start()
				# draw an alarm on the frame
				cv.putText(frame, "DROWSINESS ALERT!", (10, 30),cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
		# otherwise, the eye aspect ratio is not below the blink
		# threshold, so reset the counter and alarm
		else:
			COUNTER = 0
			alarm_status = False

		if My_mouth_MAR>=MAR_threshold:
			COUNTER_yawn+=1
			if COUNTER_yawn>=Mouth_frames_threshold:
				if not alarm_status_yawn:
					alarm_status_yawn=True
					alarm()
					cv.putText(frame, "YAWN ALERT!", (10, 30),cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 3)
		else:
			COUNTER_yawn=0
			alarm_status_yawn=False

		cv.putText(frame, "EAR: {:.2f}".format(ear_tot), (300, 30), cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
		cv.putText(frame, "MAR: {:.2f}".format(My_mouth_MAR), (30, 30), cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
 
	# show the frame
	cv.imshow("Frame", frame)
	key = cv.waitKey(1) & 0xFF
 
	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break
# do a bit of cleanup
cv.destroyAllWindows()
#vs.stop()